### taeyoonahn

**1. 프로젝트 명 : 유니콘 기업 탐색 분석**   
**2. 분석데이터 : company_nps_data.csv(원티드 국민연금 데이터)   
 2.1. 데이터특징 :   
 -회사별 데이터로 구성   
 -회사 수 : 약 5000개   
 -월별 데이터 : 직원 수 , 국민연금 보험료(인원수에 대한 상한선이 존재함)   
 -년 단위 데이터 : 매출액(천원 단위)**   
**3. 분석목표 : 원티드 국민연금 데이터의 기업데이터를 활용해 유니콘 기업으로 보이거나 예상되는 기업들 선별   
 -유니콘 기업(Unicorn)은 기업 가치가 10억 달러(=1조 원) 이상이고 창업한 지 10년 이하인 비상장 스타트업 기업을 말한다.(출처 : wikipedia.org)   
 -비상장 기업은 기업 가치 평가의 절대적인 기준이 존재하지 않으므로 <주어진 데이터를 활용해 기업의 가치를 논리적으로 평가하는 것>이 사실상 본 프로젝트의 목표라고 할 수 있음**   
**4. 데이터분석   
 4.1. 데이터전처리   
 -company_nps_data.csv 자체가 1차적인 정제가 완료된 데이터라고 판단됨**   
 **-   
 4.2. 탐색적분석(EDA)**     
 **-   
 4.3 가설 수립**   
 **-   
 4.4 가설 검증**   
 **4.5 결론 도출**

**4.1. 데이터전처리**

In [2]:
#package 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Malgun Gothic")

from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
#데이터 불러오기
#원본데이터 유지를 위한 분석 데이터셋 만들기
df_ori=pd.read_csv('company_nps_data.csv',engine = "python", encoding = "CP949")
df=df_ori.copy()
df

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151
...,...,...,...,...,...,...
99796,171439,4630622,2017,2,2402740,11
99797,239388,1045021,2018,10,1428800,9
99798,272304,181898,2018,3,392560,4
99799,362194,2100218,2018,12,874780,3


In [4]:
#데이터 정보 확인
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99801 entries, 0 to 99800
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   회사ID      99801 non-null  int64
 1   연매출액      99801 non-null  int64
 2   년도        99801 non-null  int64
 3   월         99801 non-null  int64
 4   월별_연금보험료  99801 non-null  int64
 5   월별_직원수    99801 non-null  int64
dtypes: int64(6)
memory usage: 4.6 MB
None


> **- 데이터 구성 컬럼  
> (1).회사ID(정수형)연매출액(정수형)    
> (2).연도(정수형)   
> (3).월(정수형)    
> (4).월별 연금보험료(정수형)   
> (5).월별 직원수(정수형)**

> **- 도출 가능한 정보(파생변수) 예상    
> (1).회사별 직원수 변화율       
> (2).회사별 연매출액 변화율      
> (3).회사별 임금보험료 변화율       
> (4).회사별 연매출액/연임금보험료총액 변화율   
> (5).회사별 임금보험료/직원수 변화율**

In [6]:
#데이터 결측치 확인
df.isnull()
df.isnull().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
dtype: int64

In [ ]:
#데이터 이상치 확인
df_

**4.2. 탐색적분석(EDA)**

In [11]:
#데이터 컬럼별 분포
#회사수
df_company=df['회사ID']
df_company.drop_duplicates(inplace=True)
df_company

0         233757
1         123464
2         342124
3         386420
4         129243
          ...   
96348     580794
96578    2256869
96888     547774
97566     413938
98769      96128
Name: 회사ID, Length: 5000, dtype: int64

> **- 본 데이터는 5000개 기업의 데이터를 대상으로 한 것을 알 수 있다**

In [40]:
#데이터 컬럼별 분포
#기간
df['월']
df['연월'] = df['년도'].astype(str)+df['월'].astype(str)+'01'
df['연월기타'] = df['연월'][0:3]+'0'+df['연월'][4:6]
df['연월']=df.연월.apply(lambda x: x=df['연월기타'] if len(x)==7 else x)
df['연월']

SyntaxError: lambda cannot contain assignment (<ipython-input-40-4457050a8262>, line 6)

In [21]:
#데이터 컬럼별 분포
#직원수
df_noe=df[['회사ID','월별_직원수']]
df_noe.groupby(df_noe.회사ID).describe()

월별_직원수                                                           \
         count         mean        std    min      25%     50%      75%   
회사ID                                                                      
63        12.0  1007.166667   7.517656  995.0  1001.25  1010.0  1012.25   
95        12.0   541.000000  10.685589  522.0   537.75   539.5   542.50   
102        8.0   752.125000  24.091715  722.0   734.75   751.5   766.00   
295        8.0   259.500000  26.891316  231.0   242.00   255.5   265.25   
414       12.0   247.583333   1.505042  245.0   246.75   248.0   248.00   
...        ...          ...        ...    ...      ...     ...      ...   
2400284    7.0     4.714286   1.603567    3.0     3.50     4.0     6.00   
2437574    1.0     3.000000        NaN    3.0     3.00     3.0     3.00   
2693684    6.0     3.000000   0.000000    3.0     3.00     3.0     3.00   
3312354    5.0     3.000000   0.000000    3.0     3.00     3.0     3.00   
5159723    1.0     5.000000        NaN    5.0     5.00     5.0     5.00   

                 
            max  
회사ID             
63       1018.0  
95        562.0  
102       785.0  
295       315.0  
414       251.0  
...         ...  
2400284     7.0  
2437574     3.0  
2693684     3.0  
3312354     3.0  
5159723     5.0  

[5000 rows x 8 columns]

In [22]:
#데이터 컬럼별 분포
#매출액
df_sales=df[['회사ID','연매출액']]
df_sales.groupby(df_sales.회사ID).describe()

연매출액                                                               \
        count          mean  std           min           25%           50%   
회사ID                                                                         
63       12.0  1.132878e+10  0.0  1.132878e+10  1.132878e+10  1.132878e+10   
95       12.0  6.569474e+08  0.0  6.569474e+08  6.569474e+08  6.569474e+08   
102       8.0  7.426589e+08  0.0  7.426589e+08  7.426589e+08  7.426589e+08   
295       8.0  8.377670e+07  0.0  8.377670e+07  8.377670e+07  8.377670e+07   
414      12.0  1.973277e+07  0.0  1.973277e+07  1.973277e+07  1.973277e+07   
...       ...           ...  ...           ...           ...           ...   
2400284   7.0  1.263406e+06  0.0  1.263406e+06  1.263406e+06  1.263406e+06   
2437574   1.0  2.140160e+05  NaN  2.140160e+05  2.140160e+05  2.140160e+05   
2693684   6.0  2.380760e+05  0.0  2.380760e+05  2.380760e+05  2.380760e+05   
3312354   5.0  1.451563e+06  0.0  1.451563e+06  1.451563e+06  1.451563e+06   
5159723   1.0  2.591898e+06  NaN  2.591898e+06  2.591898e+06  2.591898e+06   

                                     
                  75%           max  
회사ID                                 
63       1.132878e+10  1.132878e+10  
95       6.569474e+08  6.569474e+08  
102      7.426589e+08  7.426589e+08  
295      8.377670e+07  8.377670e+07  
414      1.973277e+07  1.973277e+07  
...               ...           ...  
2400284  1.263406e+06  1.263406e+06  
2437574  2.140160e+05  2.140160e+05  
2693684  2.380760e+05  2.380760e+05  
3312354  1.451563e+06  1.451563e+06  
5159723  2.591898e+06  2.591898e+06  

[5000 rows x 8 columns]

In [23]:
#데이터 컬럼별 분포
#연금보험료
df_pen=df[['회사ID','월별_연금보험료']]
df_pen.groupby(df_pen.회사ID).describe()

월별_연금보험료                                                        \
           count          mean           std          min          25%   
회사ID                                                                     
63          12.0  4.101282e+08  7.388361e+06  400339640.0  403938810.0   
95          12.0  2.148378e+08  3.514539e+06  209916320.0  211438670.0   
102          8.0  1.737695e+08  3.618873e+06  168943080.0  171928540.0   
295          8.0  7.731204e+07  7.952687e+06   71806740.0   73095315.0   
414         12.0  7.759019e+07  3.092534e+06   74117900.0   74737535.0   
...          ...           ...           ...          ...          ...   
2400284      7.0  1.055497e+06  4.113214e+05     549080.0     763110.0   
2437574      1.0  6.367400e+05           NaN     636740.0     636740.0   
2693684      6.0  4.381200e+05  0.000000e+00     438120.0     438120.0   
3312354      5.0  1.312200e+06  0.000000e+00    1312200.0    1312200.0   
5159723      1.0  1.257280e+06           NaN    1257280.0    1257280.0   

                                                
                 50%          75%          max  
회사ID                                            
63       410426390.0  416682265.0  419953960.0  
95       215815460.0  217429850.0  219997840.0  
102      173379370.0  175519605.0  178850980.0  
295       73853440.0   77166980.0   95257960.0  
414       77248690.0   80618200.0   81070500.0  
...              ...          ...          ...  
2400284     931860.0    1397860.0    1585600.0  
2437574     636740.0     636740.0     636740.0  
2693684     438120.0     438120.0     438120.0  
3312354    1312200.0    1312200.0    1312200.0  
5159723    1257280.0    1257280.0    1257280.0  

[5000 rows x 8 columns]

In [ ]:
#몇몇 회사 서택하여 데이터 흐름 살펴보기 데이터 흐름 확인

In [ ]:
#데이터 상관 관계 확인

**4.2.2(EDA)**

In [ ]:
#연금보험료 상승률


In [ ]:
**4.2. 가설수립(EDA)**